In [1]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [32]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

df_spacex = pd.read_csv(URL)

min_payload = int(df_spacex['Payload Mass (kg)'].min())
max_payload = int(df_spacex['Payload Mass (kg)'].max())

df_spacex.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [3]:
df_mod = df_spacex.loc[df_spacex['Booster Version'].str.contains('v1.1')]

In [4]:
df_mod['Payload Mass (kg)'].mean()

2534.6666666666665

In [5]:
# dropdown menu to choose from the various sites:
app = Dash(__name__)

app.layout = html.Div(children = [
        html.H1('SpaceX Launch Records Dashboard', 
                style = {'textAlign': 'center',
                         'color': 'black',
                         'font-size': 24}),
        html.P('Dropdown:'),
        dcc.Dropdown(id = 'site-dropdown',
                     options = [{'label': 'All Sites',    'value': 'ALL'},
                                {'label': 'CCAFS SCL-40', 'value': 'CCAFS SLC-40'},
                                {'label': 'CCAFS LC-40',  'value': 'CCAFS LC-40'},
                                {'label': 'VAFB SLC-4E',  'value': 'VAFB SLC-4E'},
                                {'label': 'KSC LC-39A',   'value': 'KSC LC-39A'}],
                                value = 'ALL',
                                placeholder = 'Select a site',
                                searchable = True),
        html.Br(),
        dcc.Graph(id = 'pie-chart'),
        html.Br(),
        html.P('Payload range slider (kg):'),
        dcc.RangeSlider(id = 'payload-slider', 
                        min = min_payload,
                        max = max_payload,
                        step = 1500,
                        marks = {min_payload: str(min_payload),
                                1500: '1500',
                                3000: '3000',
                                4500: '4500',
                                6000: '6000',
                                7500: '7500',
                                9000: '9000',
                                max_payload: str(max_payload)},
                        value = [min_payload, max_payload]),
        html.Br(),
        dcc.Graph(id = 'payload-scatter-chart')
                                    ])

Through adding the callback function, Dash will automatically be called and the output will be varied depending on what the user inputs.

In [ ]:
# function decorator to specify function input and output
@app.callback(Output('pie-chart', 'figure'),
              Input('site-dropdown', 'value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(df_spacex, values = 'class', names = 'Launch Site',
                     title = 'Successful Launch Distribution by Site')
    else:
        # choose the rows that have the same launch site as the one entered
        df_filtered = df_spacex.loc[df_spacex['Launch Site'] == entered_site]
        
        fig = px.pie(df_filtered, names = 'class',
                     title = 'Launch Distribution for ' + entered_site)
    return fig

@app.callback(Output('payload-scatter-chart', 'figure'),
              [Input('site-dropdown', 'value'),
               Input('payload-slider', 'value')])

def get_scatter_chart(entered_site, payload):
    # expand equals True to return a data frame
    # with seperated strings in different columns
    df_split = df_spacex['Booster Version'].str.split(' ', expand = True)

    df_mod = df_spacex.drop(columns = ['Booster Version'], inplace = False)
    # save the splitted column in the modified data frame
    df_mod['Booster Version'] = df_split[1]
    # based on the input payload range choose the corresponding rows
    df_mod = df_mod.loc[df_mod['Payload Mass (kg)'].between(payload[0], payload[1])]

    if entered_site == 'ALL':
        fig = px.scatter(df_mod,
                         x = 'Payload Mass (kg)', 
                         y = 'class',
                         color = 'Booster Version',
                         hover_data = ['Booster Version']) 
    else:
        df_filtered = df_mod.loc[df_mod['Launch Site'] == entered_site]
        
        fig = px.scatter(df_filtered,
                         x = 'Payload Mass (kg)', 
                         y = 'class',
                         color = 'Booster Version',
                         hover_data = ['Booster Version'])         
    return fig
    
if __name__ == '__main__':
    app.run_server(debug = True, use_reloader = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


1. Which site has the most successful launches?
2. Which site has the highest launch success rate?
3. Which payload range(s) has the highest launch success rate?
4. Which payload range(s) has the lowest launch success rate?
5. Which F9 booster version has the highest launch success rate?

In [43]:
payload_range = [[0, 4000], [4000, 6000], [6000, 10000]]

for i in payload_range:
    class_distrib = df_spacex[df_spacex['Payload Mass (kg)'].between(i[0], i[1])].value_counts('class')
    print('Payload range: ' + str(i))
    print(class_distrib)
    print('Success rate is ' + str(100*round(class_distrib[1]/(class_distrib[0]+class_distrib[1]), 2)) + ' %')

Payload range: [0, 4000]
class
0    18
1    16
dtype: int64
Success rate is 47.0 %
Payload range: [4000, 6000]
class
0    8
1    5
dtype: int64
Success rate is 38.0 %
Payload range: [6000, 10000]
class
0    6
1    3
dtype: int64
Success rate is 33.0 %


In [67]:
df_split = df_spacex['Booster Version'].str.split(' ', expand = True)
df_mod = df_spacex.drop(columns = ['Booster Version'], inplace = False)
# save the splitted column in the modified data frame
df_mod['Booster Version'] = df_split[1]
    
booster_version = df_mod['Booster Version'].unique()

for i in booster_version:
    print('Booster version: ' + i)
    class_distrib = df_mod[df_mod['Booster Version'] == i].value_counts('class')
    print(class_distrib)
    #print('Success rate is ' + str(class_distrib[1]/(class_distrib[0] + class_distrib[1])) + ' %')

Booster version: v1.0
class
0    5
dtype: int64
Booster version: v1.1
class
0    14
1     1
dtype: int64
Booster version: FT
class
1    16
0     8
dtype: int64
Booster version: B4
class
1    6
0    5
dtype: int64
Booster version: B5
class
1    1
dtype: int64
